<a href="https://colab.research.google.com/github/SNEHA-67/comp215/blob/main/labs/lab02-review.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

COMP 215 - LAB 2
----------------
#### Name(s): Sneha & Manmeet Sharma
#### Date: 15-01-25

By the end of this lab you should be able to:
  * use strings, tuples, lists, and dictionaries (review)
  * use *f-string* to simplify string formatting operations
  * write functions and simple unit tests (review)
  * use *list comprehension* to implement map and filter algorithms


During this lab, you will be introduced to the following:
  * API Keys
  * JSON data formats
  * the `datetime` module to represent dates and times
  * the `pprint` module to format data structures

## Near Earth Objects

In this lab, we'll answer some questions about [Near Earth Objects](https://cneos.jpl.nasa.gov/) using NASA's API:  [https://api.nasa.gov/](https://api.nasa.gov/#NeoWS).  But, as usual, let's do the imports first.


In [5]:
import datetime, json, requests
from pprint import pprint    # Pretty Print - built-in python function to nicely format data structures

### dates & f-strings

First we need a short tutorial on python dates and [f-strings](https://realpython.com/python-f-strings/)...

In [6]:
today = datetime.date.today()   # get a date object representing today's date
print(today, type(today))
formatted_date = f'Today is: {today}'   # A format string - notice how the variable `today` is formatted into the string
print(formatted_date)

2025-01-16 <class 'datetime.date'>
Today is: 2025-01-16


### Make an API Query

Let's get some data from the NEO database; here's a query that gets the observation "feed" for today.  Notice the first line of the block is an 'API key'.  An API key is a unique identifier that authenticates a user when making a request to an API.  Run the block below using the API key 'DEMO_KEY' (just to be sure it works), then go to https://api.nasa.gov and get your own API key.  Use your own API key for the remainder of the lab.

In [19]:
import requests, json, pprint
from pprint import pprint
API_KEY = 'DEMO_KEY'  # substitute your API key here

# get today's date (as a string)
#today = str(datetime.date.today())
today = '2023-01-13'

# use an f-string to "format" the date and API key varaibles.
url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={today}&end_date={today}&api_key={API_KEY}'

# make the API request (recall from lab 1)
response = requests.request("GET",url, headers={}, data={})

# convert the response to json format (recall from lab 1)
data = json.loads(response.text)

# TIP: print(data) to see the whole data structure returned, here we grab just the list of NEO's:
n_results = data['element_count']
neos = data['near_earth_objects'][today]
print(f'{n_results} Near Earth Objects found for {today}')
pprint(neos[:1])

16 Near Earth Objects found for 2023-01-13
[{'absolute_magnitude_h': 18.63,
  'close_approach_data': [{'close_approach_date': '2023-01-13',
                           'close_approach_date_full': '2023-Jan-13 15:15',
                           'epoch_date_close_approach': 1673622900000,
                           'miss_distance': {'astronomical': '0.0851878685',
                                             'kilometers': '12743923.677440095',
                                             'lunar': '33.1380808465',
                                             'miles': '7918706.977839511'},
                           'orbiting_body': 'Earth',
                           'relative_velocity': {'kilometers_per_hour': '27432.9512143943',
                                                 'kilometers_per_second': '7.6202642262',
                                                 'miles_per_hour': '17045.769047633'}}],
  'estimated_diameter': {'feet': {'estimated_diameter_max': 3664.5869441533,
       

Next we extract just the potentially hazerdous asteroids, using a Comp115-style list accumulator *loop*:

In [20]:
hazards =  []
for item in neos:
  if item['is_potentially_hazardous_asteroid'] is True:
    hazards.append(item)
print(f'{len(hazards)} potentially hazardous asteroids identified.')

1 potentially hazardous asteroids identified.


### Fetch Complete Data for One Asteroid

Notice that the record for each `neo` is a dictionary with `id` field that uniquely identifies this record in the database.  We can use this `id` to fetch complete orbital and close approach data for the NEO.  For example, this query fetches the complete data set for the first hazardous asteroid...  Notice that the `miss_distance` field contains the distance (in various units) by which the NEO missed an "orbiting body".


In [21]:
id = hazards[0]['id']
url = f'https://api.nasa.gov/neo/rest/v1/neo/{id}?api_key={API_KEY}'
response = requests.request("GET", url, headers={}, data={})
data = json.loads(response.text)

pprint(data)

{'absolute_magnitude_h': 19.76,
 'close_approach_data': [{'close_approach_date': '1903-07-17',
                          'close_approach_date_full': '1903-Jul-17 16:15',
                          'epoch_date_close_approach': -2097301500000,
                          'miss_distance': {'astronomical': '0.1095194917',
                                            'kilometers': '16383882.681802679',
                                            'lunar': '42.6030822713',
                                            'miles': '10180472.6236838102'},
                          'orbiting_body': 'Venus',
                          'relative_velocity': {'kilometers_per_hour': '20557.4979152918',
                                                'kilometers_per_second': '5.7104160876',
                                                'miles_per_hour': '12773.6297463101'}},
                         {'close_approach_date': '1903-09-14',
                          'close_approach_date_full': '1903-Sep-14 22:02'

## Exercise 1

In the code cell below, **re-write the accumulator loop** that creates the list of hazards as a [list comprehension](https://realpython.com/lessons/list-comprehensions-overview/) that implements a ["filter"](https://youtu.be/hUes6y2b--0)
Notice how this provides a concise way to "filter" items of interest from a larger data set.

In [22]:
# Ex. 1 your code here
# Using a list comprehension to filter hazardous asteroids
hazards = [item for item in neos if item['is_potentially_hazardous_asteroid'] is True]

print(f'{len(hazards)} potentially hazardous asteroids identified.')


1 potentially hazardous asteroids identified.




## Exercise 2

In the code cell below, write a python function that takes a list of "close approach data" as a parameter,
and returns a 2-tuple with the (date, miss km) of the closest approach to Earth in the list (where "miss km" is the miss distance in km).

Hints:
* notice the input is a list of dictionaries.  Each dictionary has a 'close_approach_date", "orbiting_body", and 'miss_distance' field.
* we are only interested in the closest approach to "Earth"
* use a loop if that is easier to understand - we will look at more compact algorithms to solve this problem in class.

Add at least one unit test to check your work - note the test data only needs dictionaries with the fields your function actually uses.


In [23]:
# Ex. 2 your code here
def closest_approach(data):
    """
    Finds the closest approach to Earth in the given list of close approach data.

    Parameters:
        data (list): A list of dictionaries, where each dictionary contains
                     'close_approach_date', 'orbiting_body', and 'miss_distance'.

    Returns:
        tuple: A 2-tuple (date, miss_km) representing the date and miss distance of the closest approach.
    """
    closest_date = None
    closest_miss_km = float('inf')  # Start with an infinitely large distance

    for entry in data:
        # Check if the orbiting body is Earth
        if entry['orbiting_body'] == 'Earth':
            # Convert the miss distance to a float for comparison
            miss_km = float(entry['miss_distance']['kilometers'])
            if miss_km < closest_miss_km:
                closest_miss_km = miss_km
                closest_date = entry['close_approach_date']

    return closest_date, closest_miss_km

# Unit Test
test_data = [
    {'close_approach_date': '2023-01-09', 'orbiting_body': 'Earth', 'miss_distance': {'kilometers': '384400'}},
    {'close_approach_date': '2023-01-10', 'orbiting_body': 'Mars', 'miss_distance': {'kilometers': '500000'}},
    {'close_approach_date': '2023-01-11', 'orbiting_body': 'Earth', 'miss_distance': {'kilometers': '350000'}},
    {'close_approach_date': '2023-01-12', 'orbiting_body': 'Earth', 'miss_distance': {'kilometers': '400000'}}
]

# Expected closest approach is the one with '2023-01-11' and distance 350,000 km
result = closest_approach(test_data)
print("Closest approach:", result)

# Output: ('2023-01-11', 350000.0)


Closest approach: ('2023-01-11', 350000.0)


## Challenge Exercise

In the code cell below, write a complete program that:
 1. fetches the list of NEO's for this week.
 2. for each NEO, fetch it's complete orbital data and determine its closest approach to Earth
 3. identify which NEO from this week's data makes the closet approach to earth
 4. print a nice message with information about the NEO, which it will approach the Earth, and how close it will come.

Hints:
* you'll need the start and end date - end date is today, see if you can use a [`timedelta`](https://docs.python.org/3/library/datetime.html#timedelta-objects) object to ge the start date (you can do basic "date math" with `timedelta` and `date` objects!)
* you may need to modify the function we wrote in Ex. 2 to return a triple with the NEO's id included;
* lots of opportunity here for more practice with list comprehensions


In [24]:
# Ex. 3 (challenge) your code here
import datetime
import requests
import json

# NASA API Key (replace 'DEMO_KEY' with your own API key if available)
API_KEY = 'DEMO_KEY'

# Calculate the date range for the week
end_date = datetime.date.today()
start_date = end_date - datetime.timedelta(days=7)

# Format the start and end dates as strings
start_date_str = str(start_date)
end_date_str = str(end_date)

# Function to fetch NEO data for the week
def fetch_weekly_neos(start_date, end_date):
    """
    Fetches the list of Near-Earth Objects (NEOs) for the given date range.
    """
    url = f'https://api.nasa.gov/neo/rest/v1/feed?start_date={start_date}&end_date={end_date}&api_key={API_KEY}'
    response = requests.get(url)
    data = response.json()
    neos = []
    for date in data['near_earth_objects']:
        neos.extend(data['near_earth_objects'][date])
    return neos

# Function to get the closest approach for a single NEO
def closest_approach(neo_id):
    """
    Fetches the complete orbital data for a specific NEO and finds its closest approach to Earth.
    Returns a tuple (NEO ID, closest date, miss distance in km).
    """
    url = f'https://api.nasa.gov/neo/rest/v1/neo/{neo_id}?api_key={API_KEY}'
    response = requests.get(url)
    neo_data = response.json()

    # Ensure 'close_approach_data' exists
    if 'close_approach_data' not in neo_data or not neo_data['close_approach_data']:
        return neo_id, None, float('inf')  # Return no data if the field is missing

    closest_date = None
    closest_miss_km = float('inf')

    # Loop through all close approach data
    for approach in neo_data['close_approach_data']:
        if approach['orbiting_body'] == 'Earth':
            miss_km = float(approach['miss_distance']['kilometers'])
            if miss_km < closest_miss_km:
                closest_miss_km = miss_km
                closest_date = approach['close_approach_date']

    return neo_id, closest_date, closest_miss_km

    # Find closest approach for each NEO
closest_approaches = []
for neo in neos:
    neo_id = neo['id']
    neo_name = neo['name']
    neo_data = closest_approach(neo_id)
    closest_approaches.append((neo_name, *neo_data))

# Identify the NEO with the closest approach to Earth
closest_neo = min(closest_approaches, key=lambda x: x[3])  # Compare by miss distance (index 3)

# Print a nice message with the result
print("\nClosest Approach This Week:")
print(f"NEO Name: {closest_neo[0]}")
print(f"NEO ID: {closest_neo[1]}")
print(f"Closest Approach Date: {closest_neo[2]}")
print(f"Miss Distance (km): {closest_neo[3]:,.2f}")




Closest Approach This Week:
NEO Name: (2021 GU3)
NEO ID: 54135432
Closest Approach Date: 2021-04-02
Miss Distance (km): 1,121,555.43
